In [160]:
# Amazon Repatriation Automation
## Jamie needs to concatenate sales data received from markets into one excel file
#### Concatenate ICLA and Classical Sheet into one excel
##### Jamie needs to send over Cost Feed from Amazon, Concatenated Sales Feed from markets, ICLA exceptions / Classical Sheet and International Catalogue
### remove all currency symbols from cost data
###### create dashboard from the data

In [3]:
#importing libraries

import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from google.cloud import bigquery


In [4]:
#importing sales data

sales = pd.read_excel('G:\Amazon Repatriation May 2020\Amazon Repat - Amazon Data April 2.xlsx', dtype={'ean': str})

In [163]:
#importing cost data

costs = pd.read_excel("G:\Amazon Repatriation May 2020\Copy of Amazon Cost Feed Combined - April 2.xlsx", dtype={'UPC': str })

In [164]:
cols = ['UPC', 'Region']
costs[cols] = costs[cols].apply(lambda x: x.str.strip())

In [165]:
### reduce table size 
costs2 = costs[['Region','UPC','Transfer Price','Dealer Price','Net Unit Price','Currency ']]

In [167]:
### amount of rows in cost data
costs2.shape

(62380, 6)

In [168]:
#keeping any rows with Transfer Price below .30 euros
costs3 = costs2[costs2['Transfer Price'] >= .3] 

In [169]:
### amount of rows in cost data
costs3.shape

(61425, 6)

In [170]:
###check for duplicates rows
sum(costs3.duplicated())

7098

In [171]:
##drop duplicates
costs4 = costs3.drop_duplicates(ignore_index = True)

In [172]:
###check for duplicates rows
sum(costs4.duplicated())

0

In [176]:
cols1 = ['inventory_owner_group', 'vendor_group','ean']
sales[cols1] = sales[cols1].apply(lambda x: x.str.strip())

In [177]:
sales.shape

(335395, 10)

In [178]:
#concatenating inventrory owner groups, vendor groups for streamlined data

sales.loc[sales['inventory_owner_group'] == 'UK', 'inventory_owner_group_combined'] = 'UK'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.co.uk', 'inventory_owner_group_combined'] = 'UK'
sales.loc[sales['inventory_owner_group'] == 'DE', 'inventory_owner_group_combined'] = 'DE'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.de', 'inventory_owner_group_combined'] = 'DE'
sales.loc[sales['inventory_owner_group'] == 'IT', 'inventory_owner_group_combined'] = 'IT'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.it', 'inventory_owner_group_combined'] = 'IT'
sales.loc[sales['inventory_owner_group'] == 'FR', 'inventory_owner_group_combined'] = 'FR'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.fr', 'inventory_owner_group_combined'] = 'FR'
sales.loc[sales['inventory_owner_group'] == 'ES', 'inventory_owner_group_combined'] = 'ES'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.es', 'inventory_owner_group_combined'] = 'ES'
sales.loc[sales['inventory_owner_group'] == 'Pan-EU SCO', 'inventory_owner_group_combined'] = 'Pan-EU SCO'
sales.loc[sales['vendor_group'] == 'UMG UK', 'vendor_group2'] = 'UK'
sales.loc[sales['vendor_group'] == 'UMG DE', 'vendor_group2'] = 'DE'
sales.loc[sales['vendor_group'] == 'UMG FR', 'vendor_group2'] = 'FR'
sales.loc[sales['vendor_group'] == 'UMG IT', 'vendor_group2']  ='IT'
sales.loc[sales['vendor_group'] == 'UMG ES', 'vendor_group2'] ='ES'
sales.loc[sales['vendor_group'] == 'UMG AT', 'vendor_group2'] ='AT'
sales.loc[sales['vendor_group'] == '3rd Party', 'vendor_group2'] = '3rd Party'

In [179]:
#creating movement key

sales['movement'] = sales['vendor_group'] + ' to ' +sales['inventory_owner_group_combined'] 

In [181]:
sales.shape

(335395, 13)

In [5]:
#splitting up strings of UPC Amazon delivers to us

new = sales['ean'].str.split("/", n =-1, expand = True) 
sales["ean1"]= new[0] 
sales["ean2"]= new[1] 
sales["ean3"]= new[2] 
sales["ean4"]= new[3] 
sales["ean5"]= new[4] 
sales.drop(columns =["ean"], inplace = True) 

In [185]:
sales['ean1'] = sales['ean1'].str.lstrip('0')

In [186]:
sales['ean1'] = sales['ean1'].str.strip()

In [187]:
sales['ean2'] = sales['ean2'].str.lstrip(' 0')

In [191]:
sales['ean2'] = sales['ean2'].str.strip()

In [192]:
sales['ean3'] = sales['ean3'].str.lstrip(' 0')

In [193]:
sales['ean3'] = sales['ean3'].str.strip()

In [194]:
sales['ean4'] = sales['ean4'].str.lstrip(' 0')

In [195]:
sales['ean4'] = sales['ean4'].str.strip()

In [196]:
sales['ean5'] = sales['ean5'].str.lstrip(' 0')

In [197]:
sales['ean5'] = sales['ean5'].str.strip()

In [199]:
### merge data
combo1 = sales.merge(costs4, how='inner', left_on=["ean1","vendor_group2"], right_on=["UPC","Region"])

In [203]:
### merge data
combo2 = sales.merge(costs4, how='inner', left_on=["ean2","vendor_group2"], right_on=["UPC","Region"])

In [204]:
### merge data
combo3 = sales.merge(costs4, how='inner', left_on=["ean3","vendor_group2"], right_on=["UPC","Region"])

In [205]:
### merge data
combo4 = sales.merge(costs4, how='inner', left_on=["ean4","vendor_group2"], right_on=["UPC","Region"])

In [206]:
### merge data
combo5 = sales.merge(costs4, how='inner', left_on=["ean5","vendor_group2"], right_on=["UPC","Region"])

In [207]:
frames = [combo1,combo2,combo3,combo4,combo4]

combo6 = pd.concat(frames)

In [210]:
sum(combo6.duplicated())

2198

In [211]:
##drop duplicates
combo7 = combo6.drop_duplicates(ignore_index = True)

In [213]:
###amount of rows
combo7.shape

(185593, 23)

In [214]:
### drop row which didn't match
combo8 = combo7[combo7['UPC'].notna()]

In [215]:
### amount of rows
combo8.shape

(185593, 23)

In [216]:
###check for duplicates rows
sum(combo8.duplicated())

0

In [217]:
### merge data
combo9 = combo8.merge(costs4, how='inner', left_on=["ean1","inventory_owner_group_combined"], right_on=["UPC","Region"])

In [218]:
### merge data
combo10 = combo8.merge(costs4, how='inner', left_on=["ean2","inventory_owner_group_combined"], right_on=["UPC","Region"])

In [219]:
### merge data
combo11 = combo8.merge(costs4, how='inner', left_on=["ean3","inventory_owner_group_combined"], right_on=["UPC","Region"])

In [220]:
### merge data
combo12 = combo8.merge(costs4, how='inner', left_on=["ean4","inventory_owner_group_combined"], right_on=["UPC","Region"])

In [221]:
### merge data
combo13 = combo8.merge(costs4, how='inner', left_on=["ean5","inventory_owner_group_combined"], right_on=["UPC","Region"])

In [222]:
frames2 = [combo9,combo10,combo11,combo12,combo13]

combo14 = pd.concat(frames2)

In [223]:
combo14.shape

(174324, 29)

In [226]:
del combo14['Region_y']

In [227]:
del combo14['Transfer Price_y']

In [228]:
del combo14['Dealer Price_y']

In [229]:
del combo14['Net Unit Price_y']

In [230]:
del combo14['Currency _y']

In [231]:
del combo14['UPC_y']

In [233]:
## rename columns for google big query upload 
combo15 = combo14.rename(columns={'Region_x': 'Region', 'Transfer Price_x':'Transfer Price','Dealer Price_x':'Dealer Price','Net Unit Price_x':'Net Unit Price','Currency _x':'Currency ','UPC_x':'UPC'})

In [234]:
#importing international catalogue data

international_catalogue = pd.read_excel(r'G:\Amazon Repatriation May 2020\UMGI Physical Catalogue_17042020.xlsx', dtype={'UPC': str})

In [236]:
### drop redundant columns
cat2 = international_catalogue[['Rep Owner','UPC']]

In [238]:
##drop duplicates
cat3 = cat2.drop_duplicates(ignore_index = True)

In [239]:
###check for duplicates rows
sum(cat3.duplicated())

0

In [241]:
##only keep first label
cat4 = cat3.drop_duplicates(subset='UPC', keep="first")

In [243]:
### drop row which didn't match
cat5 = cat4[cat4['UPC'].notna()]

In [244]:
#merging our dataset and international catalogue to return Rep Owner for each UPC

combo_cat = pd.merge(combo15,cat5[['UPC','Rep Owner']],
                     left_on='UPC', right_on=['UPC'], how='left')

In [246]:
## rename columns 
combo_cat2 = combo_cat.rename(columns={'Rep Owner': 'Rep_Owner'})

In [248]:
### count rows and columsn in combo2
combo_cat2.shape

(174324, 24)

In [249]:
###drop caroline labels
combo_cat3 = combo_cat2[~combo_cat2.Rep_Owner.str.contains('Caroline', na=False)]

In [250]:
### check row length 
combo_cat3.shape

(170564, 24)

In [251]:
###drop caroline labels
combo_cat4 = combo_cat3[~combo_cat3.Rep_Owner.str.contains('Concord', na=False)]

In [252]:
### check row length
combo_cat4.shape

(165496, 24)

In [254]:
### check row count
combo_cat4.shape

(165496, 24)

In [255]:
#importing exception list
exception_list = pd.read_excel("G:\Amazon Repatriation May 2020\exception_list.xlsx", dtype={'UPC': str})

In [256]:
#show exception list

exception_list.shape

(6166, 2)

In [258]:
##delete duplicates from exception list

exception_list.drop_duplicates(subset ="UPC", 
                     keep = False, inplace = True)

In [259]:
##show exception list
exception_list.shape

(5677, 2)

In [260]:
##only keep first label
exception_list2 = exception_list.drop_duplicates(subset='UPC', keep="first")

In [261]:
exception_list2.shape

(5677, 2)

In [262]:
#merging data with exception list to find artist exceptions

data_with_exception = pd.merge(combo_cat4,exception_list2[['UPC','Exception']],left_on="UPC", right_on=['UPC'], how='left')

In [264]:
##assigning repat percentage, changing Nan to other 50%
data_with_exception['Exception'] = data_with_exception['Exception'].fillna(.50)

In [265]:
##assigning repat percentage, classical 60%, artist exception 40%, other 50%
 
data_with_exception.loc[data_with_exception['Exception'] == .5, 'repat_%'] = .5
data_with_exception.loc[data_with_exception['Exception'] == 'Artist ICLA', 'repat_%'] = .4
data_with_exception.loc[data_with_exception['Exception'] == 'Classical', 'repat_%'] = .6

In [267]:
##change repat % column to float for equations
data_with_exception['repat_%'] = data_with_exception['repat_%'].astype(float)

In [269]:

##Repatriation equation bracket net unit price times repat% price brackets minus transfer price

data_with_exception['Repatriation'] = (data_with_exception['Net Unit Price'] * data_with_exception['repat_%']) - data_with_exception['Transfer Price']

In [271]:
##Repatriation percentage equation  bracket repatiation divded by dealer price bracket times one hundred

data_with_exception['Repatriation_Percentage_%'] = (data_with_exception['Repatriation'] / data_with_exception['Dealer Price']) *100

In [273]:
##total repatriation percentage equation stock movements times repatriation 

data_with_exception['Total_Repatriation'] = data_with_exception['unpacked_units'] * data_with_exception['Repatriation']

In [275]:
data_with_exception.shape

(165496, 29)

In [276]:
#removing any rows with negative Total_Repatriation total 

data_with_exception.drop(data_with_exception.index[data_with_exception['Total_Repatriation'] < 0], inplace = True)

In [277]:
##show data

data_with_exception.shape

(163990, 29)

In [278]:
##remove rows with any NA for Total_Repatriation

data_with_exception2 = data_with_exception[data_with_exception['Total_Repatriation'].notna()]

In [279]:
data_with_exception2.shape

(163990, 29)

In [281]:
data_with_exception2.shape

(163990, 29)

In [282]:
##remove rows with any NA for inventory owner group

data_with_exception3 = data_with_exception2[data_with_exception2['inventory_owner_group_combined'].notna()]

In [283]:
data_with_exception3.shape

(163990, 29)

In [286]:
data_with_exception3.shape

(163990, 29)

In [287]:
## rename columns for google big query upload 
data_with_exception4 = data_with_exception3.rename(columns={'Report Type': 'Report_Type', 'Transfer Price':'Transfer_Price','Dealer Price':'Dealer_Price','Net Unit Price':'Net_Unit_Price','Currency ':'Currency','repat_%':'repat_percentage','Repatriation_Percentage_%':'Repatriation_Percentage_Final'})

In [288]:
###check for duplicates rows
sum(data_with_exception4.duplicated())

1936

In [289]:
##drop duplicates
data_with_exception5 = data_with_exception4.drop_duplicates(ignore_index = True)

In [290]:
data_with_exception5.shape

(162054, 29)

In [291]:
###check for duplicates rows
sum(data_with_exception5.duplicated())

0

In [292]:
### rename cost data
data_with_exception6 = data_with_exception5.rename(columns={'Currency':'Currency_for_calcuations'})

In [294]:
### duplicate currency columns
data_with_exception6['Final_Currency'] = data_with_exception6['Currency_for_calcuations']

In [296]:
data_with_exception6.loc[data_with_exception6['Currency_for_calcuations']=='GBP', ['Total_Repatriation']] *= 1.15

In [298]:
data_with_exception6['Final_Currency'] = data_with_exception6['Final_Currency'].str.replace('GBP', 'EUR')

In [300]:
data_with_exception6.to_gbq('commercial_affairs_analytics.amazon_repatriation_june30_5_2020', 'umg-uk')

1it [00:43, 43.53s/it]
